In [2]:
"""Copied from https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries"""

import scipy.io as spio
from scipy.interpolate import interp1d
import os
import sys


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [3]:
import numpy as np
import pandas as pd

MISSION_PATH = 'D:/HyperspectralDataAll/UHI/2020-07-01-14-34-57-ArcticSeaIce-Ben-Lange/'
mat_contents = {}
mat_contents = loadmat(filename=MISSION_PATH + 'Input/PS122_4_45_20200701_1.mat')



In [4]:
from datetime import datetime, timedelta


def print_dict_tree_keys(dictionary, indent=0):
    for key, value in dictionary.items():
        print('  ' * indent + str(key))
        if isinstance(value, dict):
            print_dict_tree_keys(value, indent + 1)


# Display the keys in the nested data structure
print_dict_tree_keys(mat_contents)

class TimeData:
    def __init__(self, time = None, value = None, time_format = 'date_num'):
        if time is not None and any(time):
            if time_format == 'date_num':
                # Convert MATLAB-style datenum to Unix epoch
                converted_times = np.zeros((time.shape))

                for i, t in enumerate(time):
                    python_datetime = datetime.fromordinal(int(t)) + timedelta(days=t % 1) - timedelta(days=366)
                    # Convert to Unix timestamp
                    unix_timestamp = python_datetime.timestamp()
                    converted_times[i] = unix_timestamp
                # Convert to Unix timestamp
                self.time = converted_times

            elif time_format == 'unix':
                self.time = time
            else:
                AttributeError
        else:
            self.time = time
            
        self.value = value
    def interpolate(self, time_interp):
        self.time_interp = time_interp
        self.value_interp = interp1d(x = self.time, y = self.value, kind='nearest', fill_value='extrapolate')(x=self.time_interp)
        

class NAV:
    def __init__(self):
        # Initialized to NONE to evoke exceptions
        self.roll = TimeData()
        self.pitch = TimeData()
        self.yaw = TimeData()
        self.pos_x = TimeData()
        self.pos_y = TimeData()
        self.lon = TimeData()
        self.lat = TimeData()
        self.pos_z = TimeData()
        self.altitude = TimeData()
    def interpolate(self, time_interp):
        # Iterate through all TimeData objects in NAV and call interpolate method
        for attr_name, time_data_obj in self.__dict__.items():
            if isinstance(time_data_obj, TimeData):
                time_data_obj.interpolate(time_interp)

__header__
__version__
__globals__
ADCP
  SpotOnTime
  NMEA_Message
  field1
  field2
  field3
  field4
  field5
  field6
  field7
  field8
  field9
  field10
  field11
  field12
  field13
  field14
  field15
  PNORC
    SpotOnTime
    CellNumber
    V1
    V2
    V3
    Speed
    Direction
    AmplitudeUnits
    Amplitude1
    Amplitude2
    Amplitude3
    Correlation1
    Correlation2
  PNORI
    SpotOnTime
    InstrumentType
    NBeams
    NCells
    BlankingDistance
    CellSize
    CoordinateSystem
  PNORS
    SpotOnTime
    ErrorCode
    StatusCode
    BatteryVoltage
    SoundSpeed
    Heading
    Pitch
    Roll
    Pressure
    Temperature
    AnalogIn1
    AnalogIn2
ALTIMETER
  Altitude
  SpotOnTime
ATTENTION
  Category
  ID
  Note
  SpotOnTime
ECO
  Backscatter
  BackscatterCounts
  BackscatterWavelength
  Chlorophyll
  ChlorophyllCounts
  ChlorophyllWavelength
  FDOM
  FDOMCounts
  FDOMWavelength
  SpotOnTime
GPCTD
  AbsoluteSalinity
  Conductivity
  ConservativeTemperature
 

In [4]:
"""Cell defining all nav data of relevance"""

# For ease, read position orientation data into structs
nav = NAV()

nav.roll = TimeData(time = mat_contents['TELEMETRY']['SpotOnTime'], 
                    value = mat_contents['TELEMETRY']['Roll'])

nav.pitch = TimeData(time = mat_contents['TELEMETRY']['SpotOnTime'], 
                    value = mat_contents['TELEMETRY']['Pitch'])

nav.yaw = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['MagneticHeading'])

nav.pos_x = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['x'])

nav.pos_y = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['y'])

nav.lat = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['Latitude'])

nav.lon = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['Longitude'])

nav.pos_z = TimeData(time = mat_contents['POSITION_RENAV']['SpotOnTime'], 
                    value = mat_contents['POSITION_RENAV']['Depth'])

nav.altitude = TimeData(time = mat_contents['ALTIMETER']['SpotOnTime'], 
                    value = mat_contents['ALTIMETER']['Altitude'])

In [9]:
attent = mat_contents['ATTENTION']

print(attent)

{'Category': array(['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown',
       'Unknown'], dtype=object), 'ID': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55], dtype=uint8), 'Note': array(['marker 1', 

In [5]:
"""Retrieve the h5 data"""
import glob
import os
import spectral as sp
%matplotlib qt

h5_folder = MISSION_PATH + '/Input/H5/'
H5_PATTERN = '*.h5'


search_path_h5 = os.path.normpath(os.path.join(h5_folder, H5_PATTERN))
H5_FILE_PATHS = glob.glob(search_path_h5)

number_of_h5_files = len(H5_FILE_PATHS)

# For illustrative purposes
h5_index = 0
H5_FILE_PATH = H5_FILE_PATHS[0]

In [6]:
import h5py
"""Reader for the h5 file format in UHI context. The user provides h5 hierarchy paths as values and keys are the names given to the attributes """
class HyperspectralLite:
    def __init__(self, h5_filename, h5_tree_dict):
        with h5py.File(h5_filename, 'r', libver='latest') as self.f:
            for attribute_name, h5_hierarchy_item_path in h5_tree_dict.items():
                print(attribute_name)
                h5_item = self.f[h5_hierarchy_item_path][()]
                self.__setattr__(attribute_name, h5_item)
            
        

In [7]:
# Takes long the first time
h5_dict_read = {'radiance_cube': 'processed/radiance/dataCube',
           'hsi_frames_timestamp': 'processed/radiance/timestamp',
           'fov': 'processed/radiance/calibration/geometric/fieldOfView',
           'wavelengths' : 'processed/radiance/calibration/spectral/band2Wavelength',
           'rgb_frames_timestamp' : 'rawdata/rgb/timestamp',
           'rgb_frames' : 'rawdata/rgb/rgbFrames'}

hyp = HyperspectralLite(h5_filename=H5_FILE_PATH, h5_tree_dict=h5_dict_read)

radiance_cube
hsi_frames_timestamp
fov
wavelengths
rgb_frames_timestamp
rgb_frames


The *h5 data can let us interpolate the positions and orientations

In [8]:


nav.interpolate(time_interp=hyp.hsi_frames_timestamp)


import matplotlib.pyplot as plt

plt.plot(nav.roll.time_interp, nav.roll.value_interp)
print(nav.roll.time.max())
print((hyp.hsi_frames_timestamp.max()-nav.roll.time.max())/3600)

1593594838.272
5.373858083221648
